In [1]:
# Initialize notebook environment.
%matplotlib inline
import boto3
import botocore
import datetime
import matplotlib.pyplot as plt
import os.path
import xarray as xr

## <b>Setting Up S3 Access Using Boto</b>
We'll use boto to access the S3 bucket. Below, we'll set the bucket ID and create a resource to access it.

Note that although the bucket is public, boto requires the presence of an AWS access key and secret key to use a s3 resource. To request data anonymously, we'll use a low-level client instead.

In [2]:
era5_bucket = 'era5-pds'

# AWS access / secret keys required
# s3 = boto3.resource('s3')
# bucket = s3.Bucket(era5_bucket)

# No AWS keys required
client = boto3.client('s3', config=botocore.client.Config(signature_version=botocore.UNSIGNED))

## <b>ERA5 Data Structure on S3</b>
The ERA5 data is chunked into distinct NetCDF files per variable, each containing a month of hourly data. These files are organized in the S3 bucket by year, month, and variable name.

The data is structured as follows:

/{year}/{month}/main.nc
               /data/{var1}.nc
                    /{var2}.nc
                    /{....}.nc
                    /{varN}.nc

where year is expressed as four digits (e.g. YYYY) and month as two digits (e.g. MM). Individual data variables (var1 through varN) use names corresponding to CF standard names convention plus any applicable additional info, such as vertical coordinate.

For example, the full file path for air temperature for January 2008 is:

/2008/01/data/air_temperature_at_2_metres.nc

Note that due to the nature of the ERA5 forecast timing, which is run twice daily at 06:00 and 18:00 UTC, the monthly data file begins with data from 07:00 on the first of the month and continues through 06:00 of the following month. We'll see this in the coordinate values of a data file we download later in the notebook.

Granule variable structure and metadata attributes are stored in main.nc. This file contains coordinate and auxiliary variable data. This file is also annotated using NetCDF CF metadata conventions.

<b>We can use the paginate method to list the top level key prefixes in the bucket, which corresponds to the available years of ERA5 data.</b>

In [3]:
paginator = client.get_paginator('list_objects')
result = paginator.paginate(Bucket=era5_bucket, Delimiter='/')
for prefix in result.search('CommonPrefixes'):
    print(prefix.get('Prefix'))

1979/
1980/
1981/
1982/
1983/
1984/
1985/
1986/
1987/
1988/
1989/
1990/
1991/
1992/
1993/
1994/
1995/
1996/
1997/
1998/
1999/
2000/
2001/
2002/
2003/
2004/
2005/
2006/
2007/
2008/
2009/
2010/
2011/
2012/
2013/
2014/
2015/
2016/
2017/
2018/
2019/
QA/
cds/


<b>
Let's take a look at the objects available for a specific month using boto's list_objects_v2 method.</b>

In [4]:

keys = []
date = datetime.date(2018,1,1) # update to desired date
prefix = date.strftime('%Y/%m/')

response = client.list_objects_v2(Bucket=era5_bucket, Prefix=prefix)
response_meta = response.get('ResponseMetadata')

if response_meta.get('HTTPStatusCode') == 200:
    contents = response.get('Contents')
    if contents == None:
        print("No objects are available for %s" % date.strftime('%B, %Y'))
    else:
        for obj in contents:
            keys.append(obj.get('Key'))
        print("There are %s objects available for %s\n--" % (len(keys), date.strftime('%B, %Y')))
        for k in keys:
            print(k)
else:
    print("There was an error with your request.")

There are 20 objects available for January, 2018
--
2018/01/data/air_pressure_at_mean_sea_level.nc
2018/01/data/air_temperature_at_2_metres.nc
2018/01/data/air_temperature_at_2_metres_1hour_Maximum.nc
2018/01/data/air_temperature_at_2_metres_1hour_Minimum.nc
2018/01/data/dew_point_temperature_at_2_metres.nc
2018/01/data/eastward_wind_at_100_metres.nc
2018/01/data/eastward_wind_at_10_metres.nc
2018/01/data/integral_wrt_time_of_surface_direct_downwelling_shortwave_flux_in_air_1hour_Accumulation.nc
2018/01/data/lwe_thickness_of_surface_snow_amount.nc
2018/01/data/northward_wind_at_100_metres.nc
2018/01/data/northward_wind_at_10_metres.nc
2018/01/data/precipitation_amount_1hour_Accumulation.nc
2018/01/data/sea_surface_temperature.nc
2018/01/data/sea_surface_wave_from_direction.nc
2018/01/data/sea_surface_wave_mean_period.nc
2018/01/data/sea_surface_wind_wave_from_direction.nc
2018/01/data/significant_height_of_wind_and_swell_waves.nc
2018/01/data/snow_density.nc
2018/01/data/surface_air_pr

## <b>Downloading Files</b>
Let's download main.nc file for that month and use xarray to inspect the metadata relating to the data files.

In [5]:
metadata_file = 'main.nc'
metadata_key = prefix + metadata_file
client.download_file(era5_bucket, metadata_key, metadata_file)
ds_meta = xr.open_dataset('main.nc', decode_times=False)
ds_meta.info()

xarray.Dataset {
dimensions:
	lat = 721 ;
	lat_ocean = 361 ;
	lon = 1440 ;
	lon_ocean = 720 ;
	nv = 2 ;
	time0 = 744 ;
	time1 = 744 ;

variables:
	float32 lat(lat) ;
		lat:standard_name = latitude ;
		lat:long_name = latitude ;
		lat:units = degrees_north ;
	float64 time1(time1) ;
		time1:units = seconds since 1970-01-01 ;
		time1:standard_name = time ;
		time1:bounds = time1_bounds ;
	float64 time0(time0) ;
		time0:units = seconds since 1970-01-01 ;
		time0:standard_name = time ;
	float32 lon(lon) ;
		lon:standard_name = longitude ;
		lon:long_name = longitude ;
		lon:units = degrees_east ;
	float32 lon_ocean(lon_ocean) ;
		lon_ocean:standard_name = longitude ;
		lon_ocean:long_name = longitude ;
		lon_ocean:units = degrees_east ;
	float32 lat_ocean(lat_ocean) ;
		lat_ocean:standard_name = latitude ;
		lat_ocean:long_name = latitude ;
		lat_ocean:units = degrees_north ;
	float32 sea_surface_wave_mean_period(time0, lat_ocean, lon_ocean) ;
		sea_surface_wave_mean_period:standard_name = 

Now let's acquire data for a single variable over the course of a month. Let's download air temperature for August of 2017 and open the NetCDF file using xarray.

Note that the cell below may take some time to execute, depending on your connection speed. Most of the variable files are roughly 1 GB in size.

In [6]:
# select date and variable of interest
date = datetime.date(2017,8,1)
var = 'precipitation_amount_1hour_Accumulation'

# file path patterns for remote S3 objects and corresponding local file
s3_data_ptrn = '{year}/{month}/data/{var}.nc'
data_file_ptrn = '{year}{month}_{var}.nc'

year = date.strftime('%Y')
month = date.strftime('%m')
s3_data_key = s3_data_ptrn.format(year=year, month=month, var=var)
data_file = data_file_ptrn.format(year=year, month=month, var=var)

if not os.path.isfile(data_file): # check if file already exists
    print("Downloading %s from S3..." % s3_data_key)
    client.download_file(era5_bucket, s3_data_key, data_file)

ds = xr.open_dataset(data_file)
ds.info

<bound method Dataset.info of <xarray.Dataset>
Dimensions:                                  (lat: 721, lon: 1440, nv: 2, time1: 744)
Coordinates:
  * lon                                      (lon) float32 0.0 0.25 ... 359.75
  * lat                                      (lat) float32 90.0 89.75 ... -90.0
  * time1                                    (time1) datetime64[ns] 2017-08-01 ... 2017-08-31T23:00:00
Dimensions without coordinates: nv
Data variables:
    time1_bounds                             (time1, nv) datetime64[ns] ...
    precipitation_amount_1hour_Accumulation  (time1, lat, lon) float32 ...
Attributes:
    source:       Reanalysis
    institution:  ECMWF
    title:        ERA5 forecasts>

The ds.info output above shows us that there are three dimensions to the data: lat, lon, and time0; and one data variable: air_temperature_at_2_metres. <b>Let's inspect the coordinate values to see what they look like...</b>

In [7]:
ds.coords.values()

ValuesView(Coordinates:
  * lon      (lon) float32 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.25 359.5 359.75
  * lat      (lat) float32 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75 -90.0
  * time1    (time1) datetime64[ns] 2017-08-01 ... 2017-08-31T23:00:00)

## <b>Temperature at Specific Locations</b>
Let's create a list of various locations and plot their temperature values during the month. Note that the longitude values of the coordinates below are not given in degrees east, but rather as a mix of eastward and westward values. The data's longitude coordinate is degrees east, so we'll convert these location coordinates accordingly to match the data.

In [8]:
# # location coordinates
# locs = [
#     {'name': 'santa_monica', 'lon': -118.496245, 'lat': 34.010341},
#     {'name': 'tallinn', 'lon': 24.753574, 'lat': 59.436962},
#     {'name': 'honolulu', 'lon': -157.835938, 'lat': 21.290014},
#     {'name': 'cape_town', 'lon': 18.423300, 'lat': -33.918861},
#     {'name': 'dubai', 'lon': 55.316666, 'lat': 25.266666},
# ]

# # convert westward longitudes to degrees east
# for l in locs:
#     if l['lon'] < 0:
#         l['lon'] = 360 + l['lon']
# locs

# locations w/ lower than average rainfall
locs_lr = [
    {'name': 'el_salvador', 'lon': 13.7942, 'lat': 88.8965},
    {'name': 'honduras', 'lon': 15.2000, 'lat': 86.2419},
    {'name': 'nicaragua', 'lon': 12.8654, 'lat': 85.2072},
    {'name': 'haiti', 'lon': 18.9712, 'lat': 72.2852},
    {'name': 'se_brazil', 'lon': 20.3332, 'lat': 46.2092},
]

# locations w/ drought risk level rainfall
locs_d = [
    {'name': 'ethiopia', 'lon': 9.1450, 'lat': 40.4897},
    {'name': 'kenya', 'lon': 0.0236, 'lat': 37.9062},
    {'name': 'somalia', 'lon': 5.1521, 'lat': 46.1996},
    {'name': 'malawi', 'lon': 13.2543, 'lat': 34.3015},
    
]

# locations w/ flood risk 
locs_fr = [
    {'name': 'argentina', 'lon': 38.4161, 'lat': 63.6167},
    {'name': 'guatemala', 'lon': 15.7835, 'lat': 90.2308},
    {'name': 'peru', 'lon': 9.1900, 'lat': 75.0152},
    {'name': 'botswana', 'lon': 22.3285, 'lat': 24.6849},
    {'name': 'zimbabwe', 'lon': 19.0154 , 'lat': 29.1549},
    {'name': 'mozambique', 'lon': 18.6657, 'lat': 35.5296},
    {'name': 'south_africa', 'lon': 30.5595, 'lat': 22.9375},
]


# convert westward longitudes to degrees east
print("Locations with lower than average detected rainfall")
for l in locs_lr:
    print(l)
    if l['lon'] < 0:
        l['lon'] = 360 + l['lon']
  
print('---------------------------------------------------')
print("Locations with drought level rainfall")
for l in locs_d:
    print(l)
    if l['lon'] < 0:
        l['lon'] = 360 + l['lon']

print('---------------------------------------------------')
print("Locations with higher than average detected rainfall")
for l in locs_fr:
    print(l)
    if l['lon'] < 0:
        l['lon'] = 360 + l['lon']
        
# print(locs_fr)
# print(locs_d)
# print(locs_lr)



Locations with lower than average detected rainfall
{'name': 'el_salvador', 'lon': 13.7942, 'lat': 88.8965}
{'name': 'honduras', 'lon': 15.2, 'lat': 86.2419}
{'name': 'nicaragua', 'lon': 12.8654, 'lat': 85.2072}
{'name': 'haiti', 'lon': 18.9712, 'lat': 72.2852}
{'name': 'se_brazil', 'lon': 20.3332, 'lat': 46.2092}
---------------------------------------------------
Locations with drought level rainfall
{'name': 'ethiopia', 'lon': 9.145, 'lat': 40.4897}
{'name': 'kenya', 'lon': 0.0236, 'lat': 37.9062}
{'name': 'somalia', 'lon': 5.1521, 'lat': 46.1996}
{'name': 'malawi', 'lon': 13.2543, 'lat': 34.3015}
---------------------------------------------------
Locations with higher than average detected rainfall
{'name': 'argentina', 'lon': 38.4161, 'lat': 63.6167}
{'name': 'guatemala', 'lon': 15.7835, 'lat': 90.2308}
{'name': 'peru', 'lon': 9.19, 'lat': 75.0152}
{'name': 'botswana', 'lon': 22.3285, 'lat': 24.6849}
{'name': 'zimbabwe', 'lon': 19.0154, 'lat': 29.1549}
{'name': 'mozambique', 'lon

In [9]:
ds_locs_fr = xr.Dataset()
ds_locs_lr = xr.Dataset()
ds_locs_d = xr.Dataset()


# interate through the locations and create a dataset
# containing the temperature values for each location
for l in locs_fr:
    name = l['name']
    lon = l['lon']
    lat = l['lat']
    var_name = name

    ds2 = ds.sel(lon=lon, lat=lat, method='nearest')

    lon_attr = '%s_lon' % name
    lat_attr = '%s_lat' % name

    ds2.attrs[lon_attr] = ds2.lon.values.tolist()
    ds2.attrs[lat_attr] = ds2.lat.values.tolist()
    ds2 = ds2.rename({var : var_name}).drop(('lat', 'lon'))
    
    ds_locs_fr = xr.merge([ds_locs_fr, ds2])

ds_locs_fr.data_vars

for l in locs_d:
    name = l['name']
    lon = l['lon']
    lat = l['lat']
    var_name = name

    ds2 = ds.sel(lon=lon, lat=lat, method='nearest')

    lon_attr = '%s_lon' % name
    lat_attr = '%s_lat' % name

    ds2.attrs[lon_attr] = ds2.lon.values.tolist()
    ds2.attrs[lat_attr] = ds2.lat.values.tolist()
    ds2 = ds2.rename({var : var_name}).drop(('lat', 'lon'))
    
    ds_locs_d = xr.merge([ds_locs_d, ds2])

ds_locs_d.data_vars

for l in locs_lr:
    name = l['name']
    lon = l['lon']
    lat = l['lat']
    var_name = name

    ds2 = ds.sel(lon=lon, lat=lat, method='nearest')

    lon_attr = '%s_lon' % name
    lat_attr = '%s_lat' % name

    ds2.attrs[lon_attr] = ds2.lon.values.tolist()
    ds2.attrs[lat_attr] = ds2.lat.values.tolist()
    ds2 = ds2.rename({var : var_name}).drop(('lat', 'lon'))
    
    ds_locs_lr = xr.merge([ds_locs_lr, ds2])

ds_locs_lr.data_vars

print(ds_locs_fr.data_vars)
print(ds_locs_lr.data_vars)
print(ds_locs_d.data_vars)

Data variables:
    time1_bounds  (time1, nv) datetime64[ns] 2017-07-31T21:00:00 ... 2017-08-31T23:00:00
    argentina     (time1) float32 ...
    guatemala     (time1) float32 ...
    peru          (time1) float32 ...
    botswana      (time1) float32 ...
    zimbabwe      (time1) float32 ...
    mozambique    (time1) float32 ...
    south_africa  (time1) float32 ...
Data variables:
    time1_bounds  (time1, nv) datetime64[ns] 2017-07-31T21:00:00 ... 2017-08-31T23:00:00
    el_salvador   (time1) float32 ...
    honduras      (time1) float32 ...
    nicaragua     (time1) float32 ...
    haiti         (time1) float32 ...
    se_brazil     (time1) float32 ...
Data variables:
    time1_bounds  (time1, nv) datetime64[ns] 2017-07-31T21:00:00 ... 2017-08-31T23:00:00
    ethiopia      (time1) float32 ...
    kenya         (time1) float32 ...
    somalia       (time1) float32 ...
    malawi        (time1) float32 ...


## <b>Dataframe for Lower than Average Rainfall</b>

In [10]:
df_lr_f= ds_locs_lr.to_dataframe()
df_lr_f.describe()

,el_salvador,honduras,nicaragua,haiti,se_brazil
count,1488.000000,1488.000000,1488.000000,1488.000000,1488.000000
mean,0.000067,0.000054,0.000059,0.000081,0.000062
std,0.000129,0.000097,0.000108,0.000243,0.000316
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000061,0.000061,0.000061,0.000061,0.000000
max,0.000977,0.000671,0.000854,0.002075,0.004272


## <b>Dataframe for Drought Level Rainfall</b>

In [11]:
df_d_f = ds_locs_d.to_dataframe()
df_d_f.describe()

,ethiopia,kenya,somalia,malawi
count,1488.000000,1488.000000,1488.000000,1.488000e+03
mean,0.000004,0.000034,0.000128,6.562920e-07
std,0.000051,0.000213,0.000746,6.297007e-06
min,0.000000,0.000000,0.000000,0.000000e+00
25%,0.000000,0.000000,0.000000,0.000000e+00
50%,0.000000,0.000000,0.000000,0.000000e+00
75%,0.000000,0.000000,0.000000,0.000000e+00
max,0.001038,0.003052,0.009399,6.103516e-05


## <b>Dataframe for Flood Level Rainfall</b>

In [12]:
df_fr_f = ds_locs_fr.to_dataframe()
df_fr_f.describe()

,argentina,guatemala,peru,botswana,zimbabwe,mozambique,south_africa
count,1488.000000,1488.000000,1488.000000,1488.0,1488.0,1.488000e+03,1488.0
mean,0.000158,0.000072,0.000053,0.0,0.0,4.101825e-07,0.0
std,0.000413,0.000123,0.000146,0.0,0.0,5.908064e-06,0.0
min,0.000000,0.000000,0.000000,0.0,0.0,0.000000e+00,0.0
25%,0.000000,0.000000,0.000000,0.0,0.0,0.000000e+00,0.0
50%,0.000000,0.000000,0.000000,0.0,0.0,0.000000e+00,0.0
75%,0.000061,0.000061,0.000000,0.0,0.0,0.000000e+00,0.0
max,0.003967,0.000732,0.001282,0.0,0.0,1.220703e-04,0.0


In [13]:
df_fr_f

time1_bounds  argentina  guatemala      peru  \
nv time1                                                                     
0  2017-08-01 00:00:00 2017-07-31 21:00:00   0.000061        0.0  0.000000   
   2017-08-01 01:00:00 2017-07-31 22:00:00   0.000061        0.0  0.000000   
   2017-08-01 02:00:00 2017-07-31 23:00:00   0.000183        0.0  0.000000   
   2017-08-01 03:00:00 2017-08-01 00:00:00   0.000061        0.0  0.000061   
   2017-08-01 04:00:00 2017-08-01 01:00:00   0.000244        0.0  0.000000   
...                                    ...        ...        ...       ...   
1  2017-08-31 19:00:00 2017-08-31 19:00:00   0.000000        0.0  0.000000   
   2017-08-31 20:00:00 2017-08-31 20:00:00   0.000000        0.0  0.000000   
   2017-08-31 21:00:00 2017-08-31 21:00:00   0.000000        0.0  0.000000   
   2017-08-31 22:00:00 2017-08-31 22:00:00   0.000000        0.0  0.000000   
   2017-08-31 23:00:00 2017-08-31 23:00:00   0.000000        0.0  0.000000   

                        botswana  zimbabwe  mozambique  south_africa  
nv time1                                                              
0  2017-08-01 00:00:00       0.0       0.0         0.0           0.0  
   2017-08-01 01:00:00       0.0       0.0         0.0           0.0  
   2017-08-01 02:00:00       0.0       0.0         0.0           0.0  
   2017-08-01 03:00:00       0.0       0.0         0.0           0.0  
   2017-08-01 04:00:00       0.0       0.0         0.0           0.0  
...                          ...       ...         ...           ...  
1  2017-08-31 19:00:00       0.0       0.0         0.0           0.0  
   2017-08-31 20:00:00       0.0       0.0         0.0           0.0  
   2017-08-31 21:00:00       0.0       0.0         0.0           0.0  
   2017-08-31 22:00:00       0.0       0.0         0.0           0.0  
   2017-08-31 23:00:00       0.0       0.0         0.0           0.0  

[1488 rows x 8 columns]

In [14]:
ds_fr_mean = ds_locs_fr.mean()

#precipitation_amount_1hour_Accumulation for each of the risk areas
for m in ds_fr_mean.data_vars:
    print(ds_fr_mean)
    print('Monthly-Hourly mean: ', ds_fr_mean[m].values)

<xarray.Dataset>
Dimensions:       ()
Data variables:
    argentina     float32 0.00015833044
    guatemala     float32 7.186397e-05
    peru          float32 5.274947e-05
    botswana      float32 0.0
    zimbabwe      float32 0.0
    mozambique    float32 4.101825e-07
    south_africa  float32 0.0
Monthly-Hourly mean:  0.00015833044
<xarray.Dataset>
Dimensions:       ()
Data variables:
    argentina     float32 0.00015833044
    guatemala     float32 7.186397e-05
    peru          float32 5.274947e-05
    botswana      float32 0.0
    zimbabwe      float32 0.0
    mozambique    float32 4.101825e-07
    south_africa  float32 0.0
Monthly-Hourly mean:  7.186397e-05
<xarray.Dataset>
Dimensions:       ()
Data variables:
    argentina     float32 0.00015833044
    guatemala     float32 7.186397e-05
    peru          float32 5.274947e-05
    botswana      float32 0.0
    zimbabwe      float32 0.0
    mozambique    float32 4.101825e-07
    south_africa  float32 0.0
Monthly-Hourly mean:  5.27